In [2]:
import pandas as pd


In [3]:
import pandas as pd
import zipfile
import io

In [4]:
archive_path = "archive (2).zip"

# Define variables to hold the paths to the training and testing CSV files

In [5]:
# Define variables to hold the paths to the training and testing CSV files
training_csv_path = None
testing_csv_path = None

In [ ]:
# Check if the uploaded file is a zip file
if archive_path.endswith('.zip'):
    with zipfile.ZipFile(archive_path) as zip_ref:
        # Extract the names of all files in the zip archive
        file_names = zip_ref.namelist()
        
        # Search for CSV files and separate them into training and testing files
        for file_name in file_names:
            if file_name.endswith('.csv'):
                if 'train' in file_name.lower():
                    training_csv_path = file_name
                elif 'test' in file_name.lower():
                    testing_csv_path = file_name
        
        # Check if both training and testing CSV files were found
        if training_csv_path is None or testing_csv_path is None:
            print("Error: Both training and testing CSV files not found in the zip archive.")
        else:
            # Extract the training and testing CSV files
            with zip_ref.open(training_csv_path) as train_file, zip_ref.open(testing_csv_path) as test_file:
                # Read the CSV files into Pandas DataFrames
                train_df = pd.read_csv(train_file)
                test_df = pd.read_csv(test_file)
                print("Data loaded successfully from the zip archive.")


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [2]:
# count the number of images in the respective classes 0 - Brain Tumor and 1 - healthy 
ROOT_DIR = "BrainTumorData"
number_of_images = {}

for dir in os.listdir(ROOT_DIR):
    number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

In [3]:
number_of_images.items()

dict_items([('Brain Tumor', 2418), ('Healthey', 1879)])

In [4]:
len(os.listdir("BrainTumorData"))

2

# we will split the data such that
* 70% for Train Data
* 15% for Validation
* 15% for testing

In [11]:
# we will create a trainning folder

if not os.path.exists("./train"):
    os.mkdir("./train")

    for dir in os.listdir(ROOT_DIR):
        os.makedirs("./train/"+ dir)

        for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR,dir)), 
                                    size=(math.floor(70/100*number_of_images[dir]) - 5), 
                                    replace=False):
            O = os.path.join(ROOT_DIR,dir,img) #path
            D = os.path.join("./train", dir)
            shutil.copy(O,D)
            os.remove(O)
else:
    print("The folder exsist")


In [5]:
def dataFolder(p, split):
    if not os.path.exists("./"+p):
        os.mkdir("./"+p)

        for dir in os.listdir(ROOT_DIR):
            os.makedirs("./"+p+"./"+ dir)

            for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR,dir)), 
                                    size=(math.floor(split*number_of_images[dir]) - 5), 
                                    replace=False):
                O = os.path.join(ROOT_DIR,dir,img) #path
                D = os.path.join("./"+p, dir)
                shutil.copy(O,D)
                os.remove(O)
    else:
        print(f"{p} Folder exsist")


In [6]:
dataFolder("train",0.70)

train Folder exsist


In [7]:
dataFolder("val",0.15)


val Folder exsist


In [8]:
dataFolder("test",0.15)


test Folder exsist


In [9]:
# count the number of images in the respective classes 0 - Brain Tumor and 1 - healthy 
ROOT_DIR = "BrainTumorData"
number_of_images = {}

for dir in os.listdir(ROOT_DIR):
    number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

number_of_images.items()

dict_items([('Brain Tumor', 2418), ('Healthey', 1879)])

# Model Building

In [10]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras

In [11]:
# CNN Model

model = Sequential()

model.add(Conv2D(filters = 16, kernel_size=(3,3), activation='relu', input_shape = (224,224,3)))

model.add(Conv2D(filters = 36, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 128, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate = 0.25))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate = 0.25))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 36)      5220      
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 36)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      20800     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 128)      

In [12]:
model.compile(optimizer='adam', loss = keras.losses.binary_crossentropy, metrics=['accuracy'])

# preparing our data using Data Generator

In [13]:
def preprocessingImages(path):
    '''
    input : path
    output : pre processed images
    '''

    image_data = ImageDataGenerator(zoom_range= 0.2, shear_range= 0.2, rescale= 1/255, horizontal_flip= True)
    image = image_data.flow_from_directory(directory= path, target_size=(224,224), batch_size= 32, class_mode='binary')

    return image

In [14]:
path = "train"
train_data = preprocessingImages(path)

Found 77 images belonging to 2 classes.


In [15]:
path = "test"
test_data = preprocessingImages(path)

Found 52 images belonging to 2 classes.


In [16]:
path = "val"
val_data = preprocessingImages(path)

Found 52 images belonging to 2 classes.


In [17]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

#early stopping

es = EarlyStopping(monitor = "val_accuracy", min_delta= 0.01, patience = 3, verbose= 1, mode= 'auto')

#Model check point
mc = ModelCheckpoint(monitor = "val_accuracy", filepath = "./bestmodel.h5", verbose =1, save_best_only = True, mode = "auto")

cd = [es,mc]

# Model Train

In [18]:
hs = model.fit_generator(generator=train_data,
                         steps_per_epoch=8, 
                         epochs= 30, 
                         verbose = 1, 
                         validation_data= val_data , 
                         validation_steps= 16 , 
                         callbacks=cd)

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19068\1884689697.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hs = model.fit_generator(generator=train_data,


Epoch 1/30


3/8 [==========>...................] - ETA: 3s - loss: 0.7363 - accuracy: 0.6104WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 240 batches). You may need to use the repeat() function when building your dataset.

Epoch 1: val_accuracy improved from -inf to 0.67308, saving model to .\bestmodel.h5


C:\Users\KIIT\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 11s 731ms/step - loss: 0.7363 - accuracy: 0.6104 - val_loss: 0.6549 - val_accuracy: 0.6731
